In [32]:
import pandas as pd
import numpy as np
import math as mt
from numpy import dot
from numpy.linalg import norm
from scipy import stats
from sklearn.decomposition import TruncatedSVD 

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
import IPython
import IPython.display

sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [2]:
def euclidean_distance(A, B):
    err = 0
    length = len(A)
    for idx in range(0, length):
        tmp = ((A[idx] - B[idx]) ** 2)
        err += tmp
        
    return mt.sqrt(err)

def cosine_similarity(A,B):
    return dot(A, B) / (norm(A) * norm(B))

def sumDiffer(A,B):
    length = len(A)
    err = 0
    for idx in range(0, length):
        err += ((A[idx] - B[idx]) ** 2)
    return mt.sqrt(err / length)

def improved_similarity(A, B, w):
    cos_sim = cosine_similarity(A, B)
    sum_diff = sumDiffer(A, B)
    w **= sum_diff
    
    return cos_sim * w

## an improved collaborative recommendation algorithm based on optimizer user similarity fit test
- 랜덤하게 (100, 32) 크기의 행렬 구조 생성 100 looping
- 어떠한 벡터를 정해서 해당 벡터에 대한 순위 매기기, sort_values
---
1. Euclidean Distance
    - 거리기반 유사도 측정값을 오름차순 정렬하고, 각 순위 사이의 방향기반(Cosine Similarity) 지수가 표준편차의 n% 를 넘어가면 잘못된 측정으로 Count
2. Cosine Similarity
    - 방향기반 유사도 측정값을 내림차순 정렬하고, 각 순위 사이의 거리기반(Euclidean Distance) 지수가 표준편차의 n% 를 넘어가면 잘못된 측정으로 Count
3. imp cos
    - 거리+방향 기반 유사도 측정값을 내림차순 정렬하고, 각 순위 사이의 방향기반과 거리기반 지수가 표준편차의 n%를 넘어가면 잘못된 측정으로 Count

In [57]:
def test_func(test_per):
    fit_cols = ['execute 거리기반, cos_err', 'execute 방향기반, dis_err', 
                'execute 거리+방향기반, cos_err',
                'execute 거리+방향기반, dis_err']
    fit_df = pd.DataFrame(columns=fit_cols)

    for test_idx in range(1, 101):
        datas = np.round((np.random.rand(101,32) * 500))
        cols = ['유클리디안 거리', '코사인 유사도', '향상된 코사인 유사도']
        sim_df = pd.DataFrame(columns=cols)

        A = datas[0]
        B_values = datas[1:]
        imp_weight = 0.99

        for idx,B in enumerate(B_values):
            sim_df.loc[idx] = [
                euclidean_distance(A,B),
                cosine_similarity(A,B),
                improved_similarity(A, B, imp_weight)
            ]

        euc_std = sim_df['유클리디안 거리'].std()
        euc_std_per = euc_std * (test_per / 100)
        print("유클리디안 거리 표준편차 : {}\n".format(euc_std)
             +"유클리디안 거리 표준편차 ({}%) : {}\n".format(test_per,euc_std_per))

        cos_std = sim_df['코사인 유사도'].std()
        cos_std_per = cos_std * (test_per / 100)
        print("코사인 유사도 표준편차 : {}\n".format(cos_std)
             +"코사인 유사도 표준편차 ({}%) : {}\n".format(test_per,cos_std_per))



        sort_euc_based = sim_df.sort_values(by=['유클리디안 거리'])
        sort_cos_based = sim_df.sort_values(by=['코사인 유사도'], ascending=False)
        sort_imp_based = sim_df.sort_values(by=['향상된 코사인 유사도'], ascending=False)

        count_arr = []

        length = len(sort_euc_based)
        count = 0
        for idx in range(1, length):
            err = abs(sort_euc_based.iloc[idx - 1]['코사인 유사도']
                     - sort_euc_based.iloc[idx]['코사인 유사도']
                     )

            if err > cos_std_per:
                count += 1

        print("표준편차를 넘어선 개수 : {}\n".format(count))
        count_arr.append(count)

        length = len(sort_cos_based)
        count = 0
        for idx in range(1, length):
            err = abs(sort_cos_based.iloc[idx - 1]['유클리디안 거리']
                     - sort_cos_based.iloc[idx]['유클리디안 거리']
                     )

            if err > euc_std_per:
                count += 1

        print("표준편차를 넘어선 개수 : {}\n".format(count))
        count_arr.append(count)

        length = len(sort_imp_based)
        dis_count = 0
        cos_count = 0
        for idx in range(1, length):
            dis_err = abs(sort_imp_based.iloc[idx - 1]['유클리디안 거리']
                     - sort_imp_based.iloc[idx]['유클리디안 거리']
                     )
            cos_err = abs(sort_imp_based.iloc[idx - 1]['코사인 유사도']
                     - sort_imp_based.iloc[idx]['코사인 유사도']
                     )

            if dis_err > euc_std_per:
                dis_count += 1
            if cos_err > cos_std_per:
                cos_count += 1
        count_arr.append(cos_count)
        count_arr.append(dis_count)

        print("표준편차를 넘어선 개수 : {}".format(dis_count))
        print("표준편차를 넘어선 개수 : {}\n".format(cos_count))

        fit_df.loc['test {}'.format(test_idx)] = count_arr

        IPython.display.clear_output()
    print("compare {}% 거리기반 방향성 mean error count, {}% 거리+방향 기반 방향성 mean error count".format(test_per, test_per))
    print("거리기반 =================> {}".format(round(fit_df['execute 거리기반, cos_err'].mean())))
    print("거리 + 방향 기반 ==========> {} \n".format(round(fit_df['execute 거리+방향기반, cos_err'].mean())))


    print("compare {}% 방향기반 거리 mean error count, {}% 거리+방향 기반 거리 mean error count".format(test_per, test_per))
    print("방향기반 =================> {}".format(round(fit_df['execute 방향기반, dis_err'].mean())))
    print("거리 + 방향 기반 ==========> {} \n".format(round(fit_df['execute 거리+방향기반, dis_err'].mean())))

    fit_df

In [58]:
test_func(90)

compare 90% 거리기반 방향성 mean error count, 90% 거리+방향 기반 방향성 mean error count
거리기반 =================> 13
거리 + 방향 기반 ==========> 6 

compare 90% 방향기반 거리 mean error count, 90% 거리+방향 기반 거리 mean error count
방향기반 =================> 13
거리 + 방향 기반 ==========> 0 



In [59]:
test_func(80)

compare 80% 거리기반 방향성 mean error count, 80% 거리+방향 기반 방향성 mean error count
거리기반 =================> 15
거리 + 방향 기반 ==========> 8 

compare 80% 방향기반 거리 mean error count, 80% 거리+방향 기반 거리 mean error count
방향기반 =================> 15
거리 + 방향 기반 ==========> 0 



In [60]:
test_func(50)

compare 50% 거리기반 방향성 mean error count, 50% 거리+방향 기반 방향성 mean error count
거리기반 =================> 36
거리 + 방향 기반 ==========> 26 

compare 50% 방향기반 거리 mean error count, 50% 거리+방향 기반 거리 mean error count
방향기반 =================> 35
거리 + 방향 기반 ==========> 2 



In [61]:
test_func(25)

compare 25% 거리기반 방향성 mean error count, 25% 거리+방향 기반 방향성 mean error count
거리기반 =================> 61
거리 + 방향 기반 ==========> 51 

compare 25% 방향기반 거리 mean error count, 25% 거리+방향 기반 거리 mean error count
방향기반 =================> 62
거리 + 방향 기반 ==========> 14 



In [62]:
test_func(10)

compare 10% 거리기반 방향성 mean error count, 10% 거리+방향 기반 방향성 mean error count
거리기반 =================> 80
거리 + 방향 기반 ==========> 76 

compare 10% 방향기반 거리 mean error count, 10% 거리+방향 기반 거리 mean error count
방향기반 =================> 81
거리 + 방향 기반 ==========> 47 



In [63]:
test_func(5)

compare 5% 거리기반 방향성 mean error count, 5% 거리+방향 기반 방향성 mean error count
거리기반 =================> 91
거리 + 방향 기반 ==========> 89 

compare 5% 방향기반 거리 mean error count, 5% 거리+방향 기반 거리 mean error count
방향기반 =================> 92
거리 + 방향 기반 ==========> 74 

